# Query CSV files

Serverless Synapse SQL pool enables you to read CSV files from Azure storage (DataLake or blob storage).

## Read csv file

The easiest way to see to the content of your `CSV` file is to provide file URL to `OPENROWSET` function and specify format `CSV`. If the file is publicly available or if your Azure AD identity can access this file, you should be able to see the content of the file using the query like the one shown in the following example:

In [ ]:
select top 10 *
from openrowset(
    bulk 'https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.csv',
    format = 'csv',
    parser_version = '2.0',
    firstrow = 2 ) as rows

## Data source usage

Previous example uses full path to the file. As an alternative, you can create an external data source with the location that points to the root folder of the storage:

In [ ]:
create external data source covid
with (
     location = 'https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases'
);

Once you create a data source, you can use that data source and the relative path to the file in `OPENROWSET` function:

In [ ]:
select
    top 10 *
from
    openrowset(
        bulk 'latest/ecdc_cases.csv',
        data_source = 'covid',
        format = 'csv',
        parser_version ='2.0',
        firstrow = 2
    ) as rows

## Explicitly specify schema 

`OPENROWSET` enables you to explicitly specify what columns you want to read from the file using `WITH` clause:

In [ ]:
select
    top 10 *
from
    openrowset(
        bulk 'https://pandemicdatalake.blob.core.windows.net/public/curated/covid-19/ecdc_cases/latest/ecdc_cases.csv',
        format = 'csv',
        parser_version ='2.0',
        firstrow = 2
    ) with (
        date_rep date 1,
        cases int 5,
        geo_id varchar(6) 8
    ) as rows

The numbers after a data type in the `WITH` clause represent column index in the CSV file.